# Q-Learning

In [1]:
import random
from random import randint
import pandas as pd
import numpy as np

In [2]:
# mostra a posição
def printTable(p):
  table = pd.DataFrame(np.zeros((4,4),dtype=int),columns=None)
  table.iloc[3,3]='X'
  table.iloc[3,1]='T'
  table.iloc[0,3]='T'
  T = pd.DataFrame({'linhas':[0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3],'colunas':[0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3]})
  table = table.replace(0,'_')
  table.iloc[T['linhas'][p],T['colunas'][p]] = 'o'
  print(table.to_string(index=False,header=False))
  print('')

In [3]:
printTable(np.random.randint(0,16))

_  _  _  T
_  _  _  _
o  _  _  _
_  T  _  X



In [4]:
R = np.array([[-1,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
              [ 0, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
              [-1,  0, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
              [-1, -1,  0, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1],
              [ 0, -1, -1, -1, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1],
              [-1,  0, -1, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1],
              [-1, -1,  0, -1, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1],
              [-1, -1, -1,  0, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1, -1, -1],
              [-1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1,  0, -1, -1, -1],
              [-1, -1, -1, -1, -1,  0, -1, -1,  0, -1,  0, -1, -1,  0, -1, -1],
              [-1, -1, -1, -1, -1, -1,  0, -1, -1,  0, -1,  0, -1, -1,  0, -1],
              [-1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0, -1, -1, -1, -1, 20],
              [-1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1],
              [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0, -1,  0, -1],
              [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0, -1, 20],
              [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 50]])
Rtable = pd.DataFrame(data=R)
Rtable

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1,0,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,-1,0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,0,-1,0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,0,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1
4,0,-1,-1,-1,-1,0,-1,-1,0,-1,-1,-1,-1,-1,-1,-1
5,-1,0,-1,-1,0,-1,0,-1,-1,0,-1,-1,-1,-1,-1,-1
6,-1,-1,0,-1,-1,0,-1,0,-1,-1,0,-1,-1,-1,-1,-1
7,-1,-1,-1,0,-1,-1,0,-1,-1,-1,-1,0,-1,-1,-1,-1
8,-1,-1,-1,-1,0,-1,-1,-1,-1,0,-1,-1,0,-1,-1,-1
9,-1,-1,-1,-1,-1,0,-1,-1,0,-1,0,-1,-1,0,-1,-1


In [5]:
# fator de exploração
gamma = 1.
# taxa de aprendizado
alpha = 0.5
# zera a matriz Q
Q = np.zeros((16,16))

In [6]:
# aprendendo
def agentLearn(episodes):
    for i in range(episodes):
        s = randint(0,15)
        path = [s]
        while True:
            a = randint(0,15)
            if R[s,a]>=0:
                if a==7 or a==12:
                    print("It's a trap!")
                    path.append(a)
                    printTable(a)
                    break
                else:
                    Q[s,a] = (1.-alpha)*Q[s,a] + alpha*(R[s,a] + gamma*max(Q[a,]))
                    s = a
                    path.append(s)
                    printTable(s)
                    if s==15:
                        print("EXIT!")
                        printTable(s)
                        break
        print('Path:',path)

In [7]:
agentLearn(1)

_  _  _  T
_  _  _  _
_  o  _  _
_  T  _  X

_  _  _  T
_  _  _  _
_  _  o  _
_  T  _  X

_  _  _  T
_  _  _  _
_  _  _  o
_  T  _  X

_  _  _  T
_  _  _  o
_  _  _  _
_  T  _  X

_  _  _  T
_  _  _  _
_  _  _  o
_  T  _  X

_  _  _  T
_  _  _  o
_  _  _  _
_  T  _  X

It's a trap!
_  _  _  o
_  _  _  _
_  _  _  _
_  T  _  X

Path: [10, 6, 10, 14, 13, 14, 13, 12]


In [8]:
# verifica como a "memória" está indo
Qtable = pd.DataFrame(data=Q,dtype=int)
Qtable

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# aprendendo
def agentLearnFast(episodes):
    for i in range(episodes):
        s = randint(0,15)
        while True:
            a = randint(0,15)
            if R[s,a]>=0:
                if a==7 or a==12:
                    break
                else:
                    Q[s,a] = (1.-alpha)*Q[s,a] + alpha*(R[s,a] + gamma*max(Q[a,]))
                    s = a
                    if s==15:
                        break
    return Q

In [10]:
# zera a matriz Q
Q = np.zeros((16,16))

In [11]:
# aprendendo
Q = agentLearnFast(100)
# verifica como a "memória" está indo
Qtable = pd.DataFrame(data=Q,dtype=int)
Qtable

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,52,0,0,66,0,0,0,0,0,0,0,0,0,0,0
1,38,0,53,0,64,0,0,0,0,0,0,0,0,0,0,0
2,0,51,0,32,0,69,0,0,0,0,0,0,0,0,0,0
3,0,0,43,0,0,0,0,0,0,0,0,0,0,0,0,0
4,60,0,0,0,0,74,0,0,60,0,0,0,0,0,0,0
5,0,41,0,0,65,0,60,0,0,79,0,0,0,0,0,0
6,0,0,31,0,0,68,0,0,0,0,52,0,0,0,0,0
7,0,0,0,13,0,0,17,0,0,0,0,48,0,0,0,0
8,0,0,0,0,60,0,0,0,0,74,0,0,0,0,0,0
9,0,0,0,0,0,40,0,0,64,0,83,0,0,75,0,0


In [12]:
# saindo do labirinto
def agentExit(Q):
  n = 0
  nmax = 30
  s = randint(0,15)
  while True:
    ns = random.choice(list(filter(lambda kv: kv[1] == max(Q[s,]), enumerate(Q[s,]))))[0]
    if R[s,ns]>=0:
        s = ns
        n +=1
        print(s)
        if s == 7 or s == 12:
            print("It's a trap!")
            break
        elif s == 15: 
            print("EXIT!")
            break
        if n == nmax: 
            print("LOST!")
            break
  print('Total steps:',n)

In [13]:
agentExit(Q)

10
14
15
EXIT!
Total steps: 3
